This notebook compares analytical and numerical evaluations of alchemical derivatives of a CH3OH molecule up to the third order in KSDFT.  
The results are summerized in the last section.

In [1]:
from pyscf import gto,scf
import numpy as np
import pyscf

pyscf.__version__

'2.2.1'

In [2]:
from aqa.FcMole import FcM_like

In [3]:
target_mol = """
C	-1.27549516	0.03357710	0.00000000
H	-2.10170657	-0.87840808	1.59107621
H	-2.10170657	-0.87840808	-1.59107621
H	-1.76544024	1.99528228	0.00000000
O	1.41253482	-0.22423026	0.00000000
H	2.19050133	1.29874374	0.00000000
"""
# Coordinates in Angstrom
# C	-0.67496302	0.01776824	0.00000000
# H	-1.11217530	-0.46483357	0.84196133
# H	-1.11217530	-0.46483357	-0.84196133
# H	-0.93423081	1.05585799	0.00000000
# O	0.74748129	-0.11865755	0.00000000
# H	1.15916347	0.68726564	0.00000000
dft_functional = "pbe0"  # "lda,vwn"
basis_set = "def2-TZVP"

### Analytical evaluation

In [4]:
%load_ext autoreload
from aqa.AP_class import APDFT_perturbator as AP

In [5]:
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
mf_nn=scf.RKS(mol_NN)
mf_nn.xc = dft_functional
mf_nn.scf()
# mf_nn.conv_tol = 1e-12
mf_nn.grids.level = 4
# mf_nn.nuc_grad_method().kernel()
ap_nn=AP(mf_nn,sites=[0,1,2,3,4,5],flag_response_property=True)

converged SCF energy = -115.629838065349


In [6]:
analytical_1st_alchemical_derivative = ap_nn.build_gradient()
analytical_2nd_alchemical_derivative = ap_nn.build_hessian()
analytical_3rd_alchemical_derivative = ap_nn.build_cubic()

Inefficient alchemical force in KS-DFT which calculates the response matrix for nuclear coordinates

In [7]:
inefficient_analytical_alchemical_force = np.zeros((6, 6, 3))
for i in range(6):
  inefficient_analytical_alchemical_force[i] = ap_nn.build_inefficient_alchemical_force(i)

Efficient alchemical force in KS-DFT which calculates the response matrix for nuclear coordinates

In [8]:
efficient_analytical_alchemical_force = np.zeros((6, 6, 3))
for i in range(6):
  efficient_analytical_alchemical_force[i] = ap_nn.build_alchemical_force(i)

Electric dipole derivatives $\partial ^2 E/ \partial Z_I \partial \mathbf{F}$

In [9]:
elec_electric_dipole_gradient_z = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='Z')
elec_electric_dipole_gradient_f = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='F')

Electric polarizability derivatives $\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F}$

In [10]:
elec_electric_pol_gradient = ap_nn.build_electric_polarizability_gradient()

### Numerical evaluation

In [11]:
%autoreload 2
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
fc_param = 0.001
fcs1 = [fc_param, 0.0, 0.0, 0.0, 0.0, 0.0]
fcs2 = [-fc_param, 0.0, 0.0, 0.0, 0.0, 0.0]
fcs3 = [0.0, fc_param, 0.0, 0.0, 0.0, 0.0]
fcs4 = [0.0, -fc_param, 0.0, 0.0, 0.0, 0.0]
fcs5 = [0.0, 0.0, fc_param, 0.0, 0.0, 0.0]
fcs6 = [0.0, 0.0, -fc_param, 0.0, 0.0, 0.0]
fcs7 = [0.0, 0.0, 0.0, fc_param, 0.0, 0.0]
fcs8 = [0.0, 0.0, 0.0, -fc_param, 0.0, 0.0]
fcs9 = [0.0, 0.0, 0.0, 0.0, fc_param, 0.0]
fcs10 = [0.0, 0.0, 0.0, 0.0, -fc_param, 0.0]
fcs11 = [0.0, 0.0, 0.0, 0.0, 0.0, fc_param]
fcs12 = [0.0, 0.0, 0.0, 0.0, 0.0, -fc_param]

##### Efficient numerical alchemical force

In [12]:
fmol1=FcM_like(mol_NN,fcs=fcs1)
fmol2=FcM_like(mol_NN,fcs=fcs2)
fmol3=FcM_like(mol_NN,fcs=fcs3)
fmol4=FcM_like(mol_NN,fcs=fcs4)
fmol5=FcM_like(mol_NN,fcs=fcs5)
fmol6=FcM_like(mol_NN,fcs=fcs6)
fmol7=FcM_like(mol_NN,fcs=fcs7)
fmol8=FcM_like(mol_NN,fcs=fcs8)
fmol9=FcM_like(mol_NN,fcs=fcs9)
fmol10=FcM_like(mol_NN,fcs=fcs10)
fmol11=FcM_like(mol_NN,fcs=fcs11)
fmol12=FcM_like(mol_NN,fcs=fcs12)
mf=scf.RKS(mol_NN)
mf1=scf.RKS(fmol1)
mf2=scf.RKS(fmol2)
mf3=scf.RKS(fmol3)
mf4=scf.RKS(fmol4)
mf5=scf.RKS(fmol5)
mf6=scf.RKS(fmol6)
mf7=scf.RKS(fmol7)
mf8=scf.RKS(fmol8)
mf9=scf.RKS(fmol9)
mf10=scf.RKS(fmol10)
mf11=scf.RKS(fmol11)
mf12=scf.RKS(fmol12)
mf.xc = dft_functional
mf1.xc = dft_functional
mf2.xc = dft_functional
mf3.xc = dft_functional
mf4.xc = dft_functional
mf5.xc = dft_functional
mf6.xc = dft_functional
mf7.xc = dft_functional
mf8.xc = dft_functional
mf9.xc = dft_functional
mf10.xc = dft_functional
mf11.xc = dft_functional
mf12.xc = dft_functional
# mf.conv_tol = 1e-12
# mf1.conv_tol = 1e-12
# mf2.conv_tol = 1e-12
# mf3.conv_tol = 1e-12
# mf4.conv_tol = 1e-12
# mf.grids.level = 6
# mf1.grids.level = 6
# mf2.grids.level = 6
# mf3.grids.level = 6
# mf4.grids.level = 6
# Without dm0=mf.init_guess_by_1e(), some SCFs do not converge.
e=mf.scf(dm0=mf.init_guess_by_1e())
e1=mf1.scf(dm0=mf1.init_guess_by_1e())
e2=mf2.scf(dm0=mf2.init_guess_by_1e())
e3=mf3.scf(dm0=mf3.init_guess_by_1e())
e4=mf4.scf(dm0=mf4.init_guess_by_1e())
e5=mf5.scf(dm0=mf5.init_guess_by_1e())
e6=mf6.scf(dm0=mf6.init_guess_by_1e())
e7=mf7.scf(dm0=mf7.init_guess_by_1e())
e8=mf8.scf(dm0=mf8.init_guess_by_1e())
e9=mf9.scf(dm0=mf9.init_guess_by_1e())
e10=mf10.scf(dm0=mf10.init_guess_by_1e())
e11=mf11.scf(dm0=mf11.init_guess_by_1e())
e12=mf12.scf(dm0=mf12.init_guess_by_1e())

converged SCF energy = -115.629838047823


/home/takafumi/program/pyscf/pyscf/gto/mole.py:1213: UserWarning: Function mol.dumps drops attribute with_rinv_at_nucleus because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -115.644573834733
converged SCF energy = -115.615105243849
converged SCF energy = -115.630963349913
converged SCF energy = -115.628713835226
converged SCF energy = -115.630963349912
converged SCF energy = -115.628713835226
converged SCF energy = -115.630951406925
converged SCF energy = -115.628725775637
converged SCF energy = -115.652198505842
converged SCF energy = -115.607481176071
converged SCF energy = -115.630851797099
converged SCF energy = -115.628825263996


In [13]:
nuc_grad = mf.nuc_grad_method().kernel()
nuc_grad1 = mf1.nuc_grad_method().kernel()
nuc_grad2 = mf2.nuc_grad_method().kernel()
nuc_grad3 = mf3.nuc_grad_method().kernel()
nuc_grad4 = mf4.nuc_grad_method().kernel()
nuc_grad5 = mf5.nuc_grad_method().kernel()
nuc_grad6 = mf6.nuc_grad_method().kernel()
nuc_grad7 = mf7.nuc_grad_method().kernel()
nuc_grad8 = mf8.nuc_grad_method().kernel()
nuc_grad9 = mf9.nuc_grad_method().kernel()
nuc_grad10 = mf10.nuc_grad_method().kernel()
nuc_grad11 = mf11.nuc_grad_method().kernel()
nuc_grad12 = mf12.nuc_grad_method().kernel()

--------------- RKS gradients ---------------
         x                y                z
0 C    -0.0284772766    -0.0062987150    -0.0000000000
1 H     0.0032256614     0.0061432523    -0.0213883992
2 H     0.0032256614     0.0061432523     0.0213883992
3 H     0.0090051332    -0.0138289974    -0.0000000000
4 O     0.0370925169     0.0668973739    -0.0000000000
5 H    -0.0240767593    -0.0590538219     0.0000000000
----------------------------------------------
--------------- RKS gradients ---------------
         x                y                z
0 C    -0.0284477847    -0.0063068244    -0.0000000000
1 H     0.0031918105     0.0061118543    -0.0213340436
2 H     0.0031918105     0.0061118543     0.0213340436
3 H     0.0089881636    -0.0137599841    -0.0000000000
4 O     0.0371472854     0.0669099429    -0.0000000000
5 H    -0.0240763460    -0.0590644971     0.0000000000
----------------------------------------------
--------------- RKS gradients ---------------
         x        

In [14]:
efficient_numerical_alchemical_force = np.zeros((6, 6, 3))
efficient_numerical_alchemical_force[0] = (nuc_grad1 - nuc_grad2) / (2 * fc_param)
efficient_numerical_alchemical_force[1] = (nuc_grad3 - nuc_grad4) / (2 * fc_param)
efficient_numerical_alchemical_force[2] = (nuc_grad5 - nuc_grad6) / (2 * fc_param)
efficient_numerical_alchemical_force[3] = (nuc_grad7 - nuc_grad8) / (2 * fc_param)
efficient_numerical_alchemical_force[4] = (nuc_grad9 - nuc_grad10) / (2 * fc_param)
efficient_numerical_alchemical_force[5] = (nuc_grad11 - nuc_grad12) / (2 * fc_param)

#### Numerical alchemical derivatives

In [15]:
numerical_1st_alchemical_derivative = np.zeros((6))
numerical_1st_alchemical_derivative[0] = (e1 - e2) / (2 * fc_param)
numerical_1st_alchemical_derivative[1] = (e3 - e4) / (2 * fc_param)
numerical_1st_alchemical_derivative[2] = (e5 - e6) / (2 * fc_param)
numerical_1st_alchemical_derivative[3] = (e7 - e8) / (2 * fc_param)
numerical_1st_alchemical_derivative[4] = (e9 - e10) / (2 * fc_param)
numerical_1st_alchemical_derivative[5] = (e11 - e12) / (2 * fc_param)

#### Numerical alchemical hardness

In [16]:
ap_nn=AP(mf,sites=[0,1,2,3,4,5])
ap_nn1=AP(mf1,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn2=AP(mf2,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn3=AP(mf3,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn4=AP(mf4,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn5=AP(mf5,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn6=AP(mf6,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn7=AP(mf7,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn8=AP(mf8,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn9=AP(mf9,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn10=AP(mf10,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn11=AP(mf11,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn12=AP(mf12,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
an = ap_nn.build_gradient()
an1 = ap_nn1.build_gradient()
an2 = ap_nn2.build_gradient()
an3 = ap_nn3.build_gradient()
an4 = ap_nn4.build_gradient()
an5 = ap_nn5.build_gradient()
an6 = ap_nn6.build_gradient()
an7 = ap_nn7.build_gradient()
an8 = ap_nn8.build_gradient()
an9 = ap_nn9.build_gradient()
an10 = ap_nn10.build_gradient()
an11 = ap_nn11.build_gradient()
an12 = ap_nn12.build_gradient()
ann = ap_nn.build_hessian()
ann1 = ap_nn1.build_hessian()
ann2 = ap_nn2.build_hessian()
ann3 = ap_nn3.build_hessian()
ann4 = ap_nn4.build_hessian()
ann5 = ap_nn5.build_hessian()
ann6 = ap_nn6.build_hessian()
ann7 = ap_nn7.build_hessian()
ann8 = ap_nn8.build_hessian()
ann9 = ap_nn9.build_hessian()
ann10 = ap_nn10.build_hessian()
ann11 = ap_nn11.build_hessian()
ann12 = ap_nn12.build_hessian()
dipole1 = ap_nn1.ref_elec_electric_dipole_moment * -1
dipole2 = ap_nn2.ref_elec_electric_dipole_moment * -1
dipole3 = ap_nn3.ref_elec_electric_dipole_moment * -1
dipole4 = ap_nn4.ref_elec_electric_dipole_moment * -1
dipole5 = ap_nn5.ref_elec_electric_dipole_moment * -1
dipole6 = ap_nn6.ref_elec_electric_dipole_moment * -1
dipole7 = ap_nn7.ref_elec_electric_dipole_moment * -1
dipole8 = ap_nn8.ref_elec_electric_dipole_moment * -1
dipole9 = ap_nn9.ref_elec_electric_dipole_moment * -1
dipole10 = ap_nn10.ref_elec_electric_dipole_moment * -1
dipole11 = ap_nn11.ref_elec_electric_dipole_moment * -1
dipole12 = ap_nn12.ref_elec_electric_dipole_moment * -1
pol1 = ap_nn1.ref_elec_electric_polarizability * -1
pol2 = ap_nn2.ref_elec_electric_polarizability * -1
pol3 = ap_nn3.ref_elec_electric_polarizability * -1
pol4 = ap_nn4.ref_elec_electric_polarizability * -1
pol5 = ap_nn5.ref_elec_electric_polarizability * -1
pol6 = ap_nn6.ref_elec_electric_polarizability * -1
pol7 = ap_nn7.ref_elec_electric_polarizability * -1
pol8 = ap_nn8.ref_elec_electric_polarizability * -1
pol9 = ap_nn9.ref_elec_electric_polarizability * -1
pol10 = ap_nn10.ref_elec_electric_polarizability * -1
pol11 = ap_nn11.ref_elec_electric_polarizability * -1
pol12 = ap_nn12.ref_elec_electric_polarizability * -1

In [17]:
numerical_2nd_alchemical_derivative = np.zeros((6, 6))
numerical_2nd_alchemical_derivative[0] = (an1 - an2) / (2 * fc_param)
numerical_2nd_alchemical_derivative[1] = (an3 - an4) / (2 * fc_param)
numerical_2nd_alchemical_derivative[2] = (an5 - an6) / (2 * fc_param)
numerical_2nd_alchemical_derivative[3] = (an7 - an8) / (2 * fc_param)
numerical_2nd_alchemical_derivative[4] = (an9 - an10) / (2 * fc_param)
numerical_2nd_alchemical_derivative[5] = (an11 - an12) / (2 * fc_param)

#### Numerical alchemical hardness derivatives

In [18]:
numerical_3rd_alchemical_derivative = np.zeros((6, 6, 6))
numerical_3rd_alchemical_derivative[0, :, :] = (ann1 - ann2) / (2 * fc_param)
numerical_3rd_alchemical_derivative[1, :, :] = (ann3 - ann4) / (2 * fc_param)
numerical_3rd_alchemical_derivative[2, :, :] = (ann5 - ann6) / (2 * fc_param)
numerical_3rd_alchemical_derivative[3, :, :] = (ann7 - ann8) / (2 * fc_param)
numerical_3rd_alchemical_derivative[4, :, :] = (ann9 - ann10) / (2 * fc_param)
numerical_3rd_alchemical_derivative[5, :, :] = (ann11 - ann12) / (2 * fc_param)

#### Inefficient numerical alchemical force

In [19]:
# mol_NN1=gto.M(atom="C 0 0 0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
# mol_NN2=gto.M(atom="C 0 0 -0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
# mol_NN3=gto.M(atom="C 0 0 0; O 0 0 2.101",unit="Bohr",basis=basis_set)
# mol_NN4=gto.M(atom="C 0 0 0; O 0 0 2.099",unit="Bohr",basis=basis_set)
# mf_nn1=scf.RKS(mol_NN1)
# mf_nn2=scf.RKS(mol_NN2)
# mf_nn3=scf.RKS(mol_NN3)
# mf_nn4=scf.RKS(mol_NN4)
# mf_nn1.xc = dft_functional
# mf_nn2.xc = dft_functional
# mf_nn3.xc = dft_functional
# mf_nn4.xc = dft_functional
# mf_nn1.scf()
# mf_nn2.scf()
# mf_nn3.scf()
# mf_nn4.scf()
# ap_nn1=AP(mf_nn1,sites=[0,1])
# ap_nn2=AP(mf_nn2,sites=[0,1])
# ap_nn3=AP(mf_nn3,sites=[0,1])
# ap_nn4=AP(mf_nn4,sites=[0,1])
# ad1 = ap_nn1.build_gradient()
# ad2 = ap_nn2.build_gradient()
# ad3 = ap_nn3.build_gradient()
# ad4 = ap_nn4.build_gradient()

In [20]:
# inefficient_numerical_alchemical_force = np.zeros((2, 2))  # only z axis components
# inefficient_numerical_alchemical_force[:, 0] = (ad1 - ad2) / (2 * fc_param)
# inefficient_numerical_alchemical_force[:, 1] = (ad3 - ad4) / (2 * fc_param)

#### Numerical alchemical electric dipole moment

In [21]:
numer_elec_electric_dipole_gradient = np.zeros((6, 3))
numer_elec_electric_dipole_gradient[0] = (dipole1 - dipole2) / (2 * fc_param)
numer_elec_electric_dipole_gradient[1] = (dipole3 - dipole4) / (2 * fc_param)
numer_elec_electric_dipole_gradient[2] = (dipole5 - dipole6) / (2 * fc_param)
numer_elec_electric_dipole_gradient[3] = (dipole7 - dipole8) / (2 * fc_param)
numer_elec_electric_dipole_gradient[4] = (dipole9 - dipole10) / (2 * fc_param)
numer_elec_electric_dipole_gradient[5] = (dipole11 - dipole12) / (2 * fc_param)

#### Numerical alchemical electric polarizability

In [22]:
numer_elec_electric_pol_gradient = np.zeros((6, 3, 3))
numer_elec_electric_pol_gradient[0] = (pol1 - pol2) / (2 * fc_param)
numer_elec_electric_pol_gradient[1] = (pol3 - pol4) / (2 * fc_param)
numer_elec_electric_pol_gradient[2] = (pol5 - pol6) / (2 * fc_param)
numer_elec_electric_pol_gradient[3] = (pol7 - pol8) / (2 * fc_param)
numer_elec_electric_pol_gradient[4] = (pol9 - pol10) / (2 * fc_param)
numer_elec_electric_pol_gradient[5] = (pol11 - pol12) / (2 * fc_param)

## Comparison of analytical and numerical evaluations

$ \partial E/\partial Z_i $

In [23]:
print(analytical_1st_alchemical_derivative)

[-14.73429536  -1.12475727  -1.12475727  -1.11281549 -22.35866411
  -1.01326651]


$ (\partial E/\partial Z_i)_\mathrm{analytical} $ - $ (\partial E/\partial Z_i)_\mathrm{numerical} $

In [24]:
print(analytical_1st_alchemical_derivative - numerical_1st_alchemical_derivative)

[8.45393977e-08 7.31566114e-08 7.27444824e-08 1.54603620e-07
 7.75386304e-07 4.33062448e-08]


$\partial^2E/\partial Z_i\partial Z_j$

In [25]:
print(analytical_2nd_alchemical_derivative)

[[-2.98293629  0.39397372  0.39397372  0.39283205  0.2975315   0.27529715]
 [ 0.39397372 -1.08949187  0.32087833  0.32602965  0.2791038   0.26650417]
 [ 0.39397372  0.32087833 -1.08949187  0.32602965  0.2791038   0.26650417]
 [ 0.39283205  0.32602965  0.32602965 -1.08691572  0.27464515  0.27361192]
 [ 0.2975315   0.2791038   0.2791038   0.27464515 -3.58626745  0.43909326]
 [ 0.27529715  0.26650417  0.26650417  0.27361192  0.43909326 -0.96544947]]


$(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{numerical}$

In [26]:
print(analytical_2nd_alchemical_derivative - numerical_2nd_alchemical_derivative)

[[-4.06529431e-06 -1.40430865e-06 -1.40431176e-06  1.53639595e-06
   8.17353401e-08  1.80893689e-06]
 [ 5.20449552e-06  1.65354309e-06  2.64328565e-07  1.30163072e-06
  -5.35677461e-06 -8.25558528e-07]
 [ 5.20450973e-06  2.64325013e-07  1.65352976e-06  1.30162850e-06
  -5.35673553e-06 -8.25558527e-07]
 [-4.36628950e-06 -1.55361377e-06 -1.55362043e-06  1.24240460e-06
   9.62548466e-07  1.63057260e-06]
 [ 9.64890223e-07  1.94666034e-07  1.94676693e-07 -8.72528958e-07
   3.77376316e-06  1.73813941e-07]
 [-4.11946705e-05 -8.68440854e-06 -8.68441254e-06 -5.34459451e-06
   3.32019981e-05  1.02025968e-05]]


$\partial^3E/\partial Z_i\partial Z_j\partial Z_k$

In [27]:
print(analytical_3rd_alchemical_derivative)

[[[-0.2278832   0.08235852  0.08235852  0.08131629  0.03559915
    0.03376261]
  [ 0.08235852  0.11526836  0.07214434  0.0734432   0.04543523
    0.04055412]
  [ 0.08235852  0.07214434  0.11526836  0.0734432   0.04543523
    0.04055412]
  [ 0.08131629  0.0734432   0.0734432   0.11349206  0.04294783
    0.03649246]
  [ 0.03559915  0.04543523  0.04543523  0.04294783  0.02660788
    0.01943406]
  [ 0.03376261  0.04055412  0.04055412  0.03649246  0.01943406
    0.02652831]]

 [[ 0.08235852  0.11526836  0.07214434  0.0734432   0.04543523
    0.04055412]
  [ 0.11526836 -0.82306866  0.06684564  0.06732702  0.05367046
    0.04364442]
  [ 0.07214434  0.06684564  0.06684564  0.03686199  0.02980385
    0.02616268]
  [ 0.0734432   0.06732702  0.03686199  0.067246    0.0289992
    0.02456298]
  [ 0.04543523  0.05367046  0.02980385  0.0289992   0.02805294
    0.02179797]
  [ 0.04055412  0.04364442  0.02616268  0.02456298  0.02179797
    0.03203281]]

 [[ 0.08235852  0.07214434  0.11526836  0.0734432

$(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{analytical}$ - $(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{numerical}$

In [28]:
print(analytical_3rd_alchemical_derivative - numerical_3rd_alchemical_derivative)

[[[ 2.14413885e-06  2.85913542e-07  2.85915369e-07  3.27183412e-08
   -7.18067638e-08 -2.81362511e-08]
  [ 2.85913542e-07  4.29789492e-07  4.09586204e-07  5.26126408e-08
    2.86142503e-08  2.38344076e-08]
  [ 2.85915369e-07  4.09586204e-07  4.29781603e-07  5.26108060e-08
    2.86142203e-08  2.38344890e-08]
  [ 3.27183412e-08  5.26126408e-08  5.26108060e-08  7.39186596e-07
    7.15877288e-08  8.80268004e-08]
  [-7.18067638e-08  2.86142503e-08  2.86142203e-08  7.15877288e-08
    1.70864122e-07 -6.79612948e-08]
  [-2.81362511e-08  2.38344076e-08  2.38344890e-08  8.80268004e-08
   -6.79612948e-08  5.96522863e-07]]

 [[ 6.98991780e-07 -4.67958904e-08 -2.40109510e-07 -4.56872119e-07
   -3.18819248e-07 -2.84772538e-07]
  [-4.67958904e-08 -7.17416423e-07 -8.81593937e-08 -4.30751458e-07
   -5.22458281e-07 -3.85508924e-07]
  [-2.40109510e-07 -8.81593937e-08 -4.53751203e-07 -3.33662520e-07
   -4.12436503e-07 -3.01526959e-07]
  [-4.56872119e-07 -4.30751458e-07 -3.33662520e-07 -6.83545603e-07
   -

$\partial^2E/\partial Z_i\partial R_j$

In [29]:
print(efficient_analytical_alchemical_force)

[[[ 2.94931942e-02 -8.11060045e-03 -3.55083016e-15]
  [-3.38587437e-02 -3.14059456e-02  5.43689196e-02]
  [-3.38587437e-02 -3.14059456e-02 -5.43689196e-02]
  [-1.69745412e-02  6.90302587e-02  4.07314044e-16]
  [ 5.47847263e-02  1.25598453e-02 -1.22742204e-18]
  [ 4.14402127e-04 -1.06682825e-02 -3.13829549e-18]]

 [[-3.59935455e-02  4.94955118e-03  6.96092823e-03]
  [-3.11653225e-02 -2.58882934e-02  3.62558114e-02]
  [ 3.12236544e-03  5.92748352e-03 -1.51528382e-02]
  [ 6.15725490e-03  1.05663460e-02 -1.34611210e-02]
  [ 6.04818688e-02  1.02225742e-02 -8.86832247e-03]
  [-2.60260670e-03 -5.77737221e-03 -5.73456519e-03]]

 [[-3.59935455e-02  4.94955118e-03 -6.96092823e-03]
  [ 3.12236544e-03  5.92748352e-03  1.51528382e-02]
  [-3.11653225e-02 -2.58882934e-02 -3.62558114e-02]
  [ 6.15725490e-03  1.05663460e-02  1.34611210e-02]
  [ 6.04818688e-02  1.02225742e-02  8.86832247e-03]
  [-2.60260670e-03 -5.77737221e-03  5.73456519e-03]]

 [[-3.09737728e-02  6.60153978e-04 -1.49986376e-15]
  [ 2.

$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [30]:
print(efficient_analytical_alchemical_force - efficient_numerical_alchemical_force)

[[[-1.34772200e-06 -4.02926122e-06  1.65246186e-12]
  [ 5.86685744e-08  1.52545705e-07 -1.92235271e-07]
  [ 5.86676879e-08  1.52546317e-07  1.92233057e-07]
  [ 5.36929136e-08  3.12906596e-07 -7.84274666e-14]
  [-8.96645332e-07 -1.30018600e-07  2.30188631e-12]
  [ 2.75324301e-07  1.08150890e-06 -1.04900505e-14]]

 [[ 1.07107561e-06  9.87061158e-08 -9.96832269e-07]
  [-1.18342974e-06  8.39749055e-07  3.70645000e-07]
  [-3.21043466e-07 -7.89751524e-08 -5.22328549e-08]
  [-2.46774465e-07  3.37907572e-07  4.42600804e-08]
  [ 3.11262431e-07  1.69178329e-07 -7.52580928e-08]
  [-4.28050674e-07  4.49660488e-08  2.21421937e-08]]

 [[ 1.07106317e-06  9.87075836e-08  9.96831105e-07]
  [-3.21038915e-07 -7.89752630e-08  5.22349652e-08]
  [-1.18342919e-06  8.39753778e-07 -3.70647668e-07]
  [-2.46776686e-07  3.37910570e-07 -4.42602210e-08]
  [ 3.11257096e-07  1.69183881e-07  7.52550049e-08]
  [-4.28048786e-07  4.49664930e-08 -2.21423141e-08]]

 [[-1.98340582e-07 -2.83896692e-06 -1.73539620e-12]
  [ 2.

$\partial^2E/\partial Z_i\partial R_j$

In [31]:
print(inefficient_analytical_alchemical_force)

[[[ 2.94931944e-02 -8.11060062e-03 -3.52111650e-15]
  [-3.38587437e-02 -3.14059456e-02  5.43689196e-02]
  [-3.38587437e-02 -3.14059456e-02 -5.43689196e-02]
  [-1.69745412e-02  6.90302588e-02  5.39013835e-16]
  [ 5.47847262e-02  1.25598454e-02  3.95553795e-17]
  [ 4.14402059e-04 -1.06682826e-02 -1.50543406e-17]]

 [[-3.59935453e-02  4.94955123e-03  6.96092828e-03]
  [-3.11653225e-02 -2.58882934e-02  3.62558114e-02]
  [ 3.12236540e-03  5.92748347e-03 -1.51528383e-02]
  [ 6.15725489e-03  1.05663460e-02 -1.34611210e-02]
  [ 6.04818687e-02  1.02225741e-02 -8.86832246e-03]
  [-2.60260666e-03 -5.77737215e-03 -5.73456521e-03]]

 [[-3.59935453e-02  4.94955123e-03 -6.96092828e-03]
  [ 3.12236540e-03  5.92748347e-03  1.51528383e-02]
  [-3.11653225e-02 -2.58882934e-02 -3.62558114e-02]
  [ 6.15725489e-03  1.05663460e-02  1.34611210e-02]
  [ 6.04818687e-02  1.02225741e-02  8.86832246e-03]
  [-2.60260666e-03 -5.77737215e-03  5.73456521e-03]]

 [[-3.09737730e-02  6.60153963e-04 -1.35930553e-15]
  [ 2.

$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [32]:
# print(inefficient_analytical_alchemical_force[:, :, 2] - inefficient_numerical_alchemical_force)

$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical,efficient}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical,inefficient}$

In [33]:
print(efficient_analytical_alchemical_force - inefficient_analytical_alchemical_force)

[[[-1.27790001e-10  1.67325265e-10 -2.97136634e-17]
  [ 7.30471239e-13 -1.90857052e-11  2.62347366e-11]
  [ 7.31248395e-13 -1.90854554e-11 -2.62313782e-11]
  [ 1.18674376e-11 -7.81535947e-11 -1.31699791e-16]
  [ 4.65880667e-11 -1.60520264e-10 -4.07828015e-17]
  [ 6.78811729e-11  1.09526263e-10  1.19160451e-17]]

 [[-1.90457095e-10 -5.22829557e-11 -5.70150593e-11]
  [ 2.18869367e-11  2.21062058e-11 -2.34066100e-11]
  [ 3.68945222e-11  4.62590660e-11  7.42250011e-11]
  [ 7.00702690e-12 -1.73533410e-11  3.91954524e-12]
  [ 1.57534208e-10  6.06447392e-11 -1.00896513e-11]
  [-3.28639546e-11 -5.93790989e-11  1.23734946e-11]]

 [[-1.90459093e-10 -5.22851762e-11  5.70203884e-11]
  [ 3.68945777e-11  4.62596593e-11 -7.42240991e-11]
  [ 2.18864926e-11  2.21094254e-11  2.34079423e-11]
  [ 7.00653077e-12 -1.73528275e-11 -3.91806032e-12]
  [ 1.57528879e-10  6.06458772e-11  1.00890685e-11]
  [-3.28640934e-11 -5.93791613e-11 -1.23734079e-11]]

 [[ 1.51500534e-10  1.48279167e-11 -1.40558232e-16]
  [-2.

$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$

In [34]:
print(elec_electric_dipole_gradient_f)

[[-8.35669955e-02 -5.07162928e-02 -4.02819143e-14]
 [-8.15774734e-01 -7.20244020e-01  1.14935467e+00]
 [-8.15774734e-01 -7.20244020e-01 -1.14935467e+00]
 [-5.34900508e-01  1.39646710e+00 -3.08450664e-14]
 [ 9.01426861e-01 -2.20952037e-01 -4.97326902e-14]
 [ 1.45969559e+00  9.74687008e-01 -3.46179962e-14]]


In [35]:
print(elec_electric_dipole_gradient_z)

[[-8.35669953e-02 -5.07162930e-02 -3.03751644e-14]
 [-8.15774734e-01 -7.20244020e-01  1.14935467e+00]
 [-8.15774734e-01 -7.20244020e-01 -1.14935467e+00]
 [-5.34900507e-01  1.39646710e+00 -3.37347130e-14]
 [ 9.01426861e-01 -2.20952037e-01 -4.40385409e-14]
 [ 1.45969559e+00  9.74687008e-01 -3.17025614e-14]]


$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$ - $(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{numerical}$

In [36]:
print(elec_electric_dipole_gradient_f - numer_elec_electric_dipole_gradient)

[[-3.06981466e-06 -1.18792143e-05  6.07664684e-12]
 [ 7.15791072e-06 -3.14276956e-06 -4.29408879e-06]
 [ 7.15786431e-06 -3.14277046e-06  4.29409754e-06]
 [-3.91976626e-06 -1.06344895e-05  4.12726215e-13]
 [-4.30074922e-06  4.43662415e-06 -1.61603230e-11]
 [-5.66423159e-05 -9.74338684e-06  1.95607097e-13]]


In [37]:
print(elec_electric_dipole_gradient_z - numer_elec_electric_dipole_gradient)

[[-3.06960189e-06 -1.18793873e-05  6.08655359e-12]
 [ 7.15755395e-06 -3.14273070e-06 -4.29438317e-06]
 [ 7.15750754e-06 -3.14273159e-06  4.29439192e-06]
 [-3.91880474e-06 -1.06348401e-05  4.09836568e-13]
 [-4.30079910e-06  4.43661288e-06 -1.61546289e-11]
 [-5.66421733e-05 -9.74330977e-06  1.98522532e-13]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$

In [38]:
print(elec_electric_pol_gradient)

[[[ 2.83814340e+00  5.38588604e-01 -8.04754077e-13]
  [ 5.38588604e-01  6.08799423e+00 -7.74607833e-13]
  [-8.04754077e-13 -7.74607833e-13  6.24740900e+00]]

 [[ 4.19188330e+00  1.08909779e+00 -9.05836312e-01]
  [ 1.08909779e+00  6.92776496e+00 -1.15320870e+00]
  [-9.05836312e-01 -1.15320870e+00  8.51705054e+00]]

 [[ 4.19188330e+00  1.08909779e+00  9.05836312e-01]
  [ 1.08909779e+00  6.92776496e+00  1.15320870e+00]
  [ 9.05836312e-01  1.15320870e+00  8.51705054e+00]]

 [[ 4.28013296e+00 -5.32721315e-02 -6.82060195e-13]
  [-5.32721315e-02  8.89199752e+00 -9.03733736e-13]
  [-6.82060195e-13 -9.03733736e-13  6.20985616e+00]]

 [[ 7.80371639e+00  9.58197020e-02 -1.11245734e-12]
  [ 9.58197020e-02  5.37194874e+00 -5.84309411e-13]
  [-1.11245734e-12 -5.84309411e-13  5.84169467e+00]]

 [[ 5.99084533e+00  4.02356376e-02 -8.67987260e-13]
  [ 4.02356376e-02  4.34561010e+00 -5.41264055e-13]
  [-8.67987260e-13 -5.41264055e-13  4.34612940e+00]]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$ - $(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{numerical}$

In [39]:
print(elec_electric_pol_gradient - numer_elec_electric_pol_gradient)

[[[-5.01202533e-06  1.48860459e-05  1.39682993e-10]
  [ 1.48860459e-05  1.56302528e-05  5.08298263e-11]
  [ 1.39682993e-10  5.08298263e-11  3.26361968e-05]]

 [[ 2.17389180e-05  2.44078674e-07  5.15139298e-07]
  [ 2.44078674e-07 -3.15078187e-05  1.13905293e-05]
  [ 5.15139298e-07  1.13905293e-05 -1.94538264e-05]]

 [[ 2.17386888e-05  2.43847032e-07 -5.14967843e-07]
  [ 2.43847032e-07 -3.15085209e-05 -1.13902762e-05]
  [-5.14967843e-07 -1.13902762e-05 -1.94542000e-05]]

 [[ 1.16353032e-05 -1.54751887e-05  3.81561356e-10]
  [-1.54751887e-05  3.50609539e-05  3.00648642e-10]
  [ 3.81561356e-10  3.00648642e-10  3.04517661e-05]]

 [[-5.40100090e-05  9.61537154e-06 -6.10728528e-10]
  [ 9.61537154e-06 -4.71913100e-05 -4.89160190e-10]
  [-6.10728528e-10 -4.89160190e-10 -1.08164566e-05]]

 [[-1.15336417e-04  3.50155018e-05  2.37990260e-11]
  [ 3.50155018e-05  1.89345916e-04  1.04160245e-10]
  [ 2.37990260e-11  1.04160245e-10  2.07352923e-04]]]
